In [2]:
import numpy as np
from astropy.io import fits
import sys
import healpy as hp
from astropy.table import Table
sys.stderr=open('error.log','w',buffering=1)

z_min=0.001
z_max=0.5
outlabel='reducedCatalogDaily.fits'

ra_column_name='RA'
dec_column_name='DEC'
z_column_name='Z_not4clus'
zerr_column_name='ZERR'
mag_column_name='OMAG'
absmag_column='AMAG'
deltachi2_column='DELTACHI2'
flux_column_name='FLUX_R'
ebv_column_name='EBV'
target_column_name='TARGETID'
weight1_column_name='FRACZ_TILELOCID'
weight2_column_name='FRAC_TLOBS_TILES'

DIR_CATALOG='/global/cfs/cdirs/desi/survey/catalogs/main/LSS/daily/LSScats/thru230708/'
glxcat='BGS_ANY_full.dat.fits'


NSIDE=1024


filename = DIR_CATALOG+glxcat
hdul= fits.open(filename,memmap=True)
h=hdul[1].data

print("Reading in fits data")
deltachi2=h[deltachi2_column]
mask_deltachi=deltachi2>40
zwarn=h['ZWARN']
zwarn_mask=(zwarn==0)
spectype=h['SPECTYPE']
spectype_mask=(spectype!='STAR')
z=h[z_column_name]
mask_z=np.logical_and(z>z_min, z<z_max)
bgs=h['BGS_TARGET']
bgs_mask=bgs>0



Reading in fits data


In [3]:
ra_g=h[ra_column_name][mask_z & mask_deltachi & zwarn_mask & spectype_mask & bgs_mask]
dec_g=h[dec_column_name][mask_z & mask_deltachi & zwarn_mask & spectype_mask & bgs_mask]
z=z[mask_z & mask_deltachi & zwarn_mask & spectype_mask & bgs_mask]
zerr_g=h[zerr_column_name][mask_z & mask_deltachi & zwarn_mask & spectype_mask & bgs_mask]
flux_g=h[flux_column_name][mask_z & mask_deltachi & zwarn_mask & spectype_mask & bgs_mask]
ebv_g=h[ebv_column_name][mask_z & mask_deltachi & zwarn_mask & spectype_mask & bgs_mask]
target_g=h[target_column_name][mask_z & mask_deltachi & zwarn_mask & spectype_mask & bgs_mask]
weight1_g=h[weight1_column_name][mask_z & mask_deltachi & zwarn_mask & spectype_mask & bgs_mask]
weight2_g=h[weight2_column_name][mask_z & mask_deltachi & zwarn_mask & spectype_mask & bgs_mask]




hdul.close()
print(ra_g)
pix_g= hp.pixelfunc.ang2pix(NSIDE, ra_g, dec_g,lonlat=True)    
t=Table([ra_g,dec_g,z,zerr_g,pix_g,flux_g,ebv_g,target_g,weight1_g,weight2_g],
        names=('TARGET_RA','TARGET_DEC','Z','ZERR', 'hpix1024','FLUX_R','EBV','TARGETID','FRACZ_TILELOCID','FRAC_TLOBS_TILES'))
t.write(outlabel,format='fits',overwrite=True)


#print('Done writing '+outlabel)

[187.22510927 187.22552437 187.22650518 ... 336.22042932 336.20280307
 336.17965185]


In [9]:
ra=h[ra_column_name]
print(len(ra))
ra=ra[zwarn_mask]
print(len(ra))
deltachi2=h[deltachi2_column][zwarn_mask]
mask_deltachi2=deltachi2>40
ra=ra[mask_deltachi2]
print(len(ra))
spectype2=h['SPECTYPE'][zwarn_mask & mask_deltachi]
spectype2_mask=(spectype2!='STAR')
ra=ra[spectype2_mask]
print(len(ra))
z2=h['Z'][zwarn_mask & mask_deltachi & spectype_mask]
mask_z2=np.logical_and(z2>z_min,z2<z_max)
ra=ra[mask_z2]
print(len(ra))
bgs2=h['BGS_TARGET'][mask_z & mask_deltachi & zwarn_mask & spectype_mask]
bgs_mask2=bgs2>0
ra=ra[bgs_mask2]
print(len(ra))


30300000
22547052
21209773
16157047
7912548
6541993


30,300,000-Original Catalog before any cuts
22,547,052-ZWARN==0 applied
21,209,773-DELTACHI2>40 applied
16,157,047-SPECTYPE !=STAR
7,912,548-0.001<z<0.5
6,541,993-BGS_TARGET>0
5,479,226-absolute magnitude cut at -21.5
4,478,235-zcut depending on cosmological models (zmin=0.03882, zmax=0.32295 for GW190412)
8,442-galaxies in 90% region


In [5]:
weight_catalog_name='/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v0.1/blinded/BGS_BRIGHT-21.5_full.dat.fits'
weight_catalog=fits.open(weight_catalog_name,memmap=True)[1].data
weight_fkp=weight_catalog['WEIGHT_FKP']
weight_comp=1/weight_catalog['FRACZ_TILELOCID']
weights_targets=weight_catalog['TARGETID']

514470
6541993
6541993


IndexError: boolean index did not match indexed array along dimension 0; dimension is 554666 but corresponding boolean dimension is 6541993